In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn

In [ ]:
pip freeze > requirements.txt

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("datasets/kaggle/startup data.csv")

In [ ]:
print("First few rows of the dataset:\n")
print("="*665)
print(df.head().to_string(index=False))
print("="*665)

print("\nData Types of Each Column:")
print("="*40)
print(df.dtypes.to_string())
print("="*40)

In [ ]:
print("\nChecking for null values:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

df = df.dropna()

print("\nUpdated Dataset Preview After Handling Missing Values:")
print("="*65)
print(df.head().to_string(index=False))
print("="*65)

# Categorical

In [ ]:
columns_to_drop = [
    'Unnamed: 6', 'closed_at', 'age_first_milestone_year', 'age_last_milestone_year', 
    'state_code.1', 'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate', 'is_software', 
    'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 
    'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 
    'is_othercategory', 'object_id', 'has_roundA', 'has_roundB', 
    'has_roundC', 'has_roundD'
]

df = df.drop(columns=columns_to_drop)

print("The specified columns have been dropped from the DataFrame.")
print("Remaining DataFrame columns:", df.columns.tolist())

In [ ]:
import os
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold

features = df.select_dtypes(include=[np.number]).drop(columns=['Unnamed: 0'])
target = df['status']

constant_filter = VarianceThreshold(threshold=0)
filtered_features = constant_filter.fit_transform(features)
filtered_feature_names = features.columns[constant_filter.get_support(indices=True)]

selector = SelectKBest(score_func=f_classif, k="all")
selected_features = selector.fit_transform(filtered_features, target)

selected_feature_names = filtered_feature_names[selector.get_support(indices=True)]
selected_feature_scores = selector.scores_[selector.get_support(indices=True)]

top_features = pd.DataFrame({'Feature': selected_feature_names, 'Score': selected_feature_scores})
top_features = top_features.sort_values(by='Score', ascending=False)

directory = "Best Features/startup data"
os.makedirs(directory, exist_ok=True)

kbest_features_path = os.path.join(directory, "categorical_kbest_features.txt")
top_features.to_string(kbest_features_path, index=False)

selected_features = top_features[top_features['Score'] > 1]
selected_kbest_features_path = os.path.join(directory, "categorical_selected_kbest_features.txt")
selected_features.to_string(selected_kbest_features_path, index=False)

print("Top features saved to:", kbest_features_path)
print("Selected features with Score > 1 saved to:", selected_kbest_features_path)

In [ ]:
# We found that `labels` column has inf importance, that is, it is corresponding to `status` target column

if 'labels' in df.columns and 'status' in df.columns:
    acquired_check = df[(df['status'] == 'acquired') & (df['labels'] != 1)]
    
    closed_check = df[(df['status'] == 'closed') & (df['labels'] != 0)]
    
    if not acquired_check.empty or not closed_check.empty:
        print("Mismatch found between 'labels' and 'status':")
        print(f"- Rows where 'status' is 'acquired' but 'labels' is not 1: {len(acquired_check)}")
        print(f"- Rows where 'status' is 'closed' but 'labels' is not 0: {len(closed_check)}")
    else:
        print("The 'labels' column correctly corresponds to the 'status'.")
else:
    print("Either 'labels' or 'status' column is missing. Please check the column names.")

In [9]:
# We have to drop `labels` as it directly corresponds to `status` target column

df.drop('labels', axis=1, inplace=True, errors='ignore')

In [ ]:
# We now drop the `labels` column from the Best Features, so that we can then proceed to save our final dataset

def remove_labels_line(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    # Filter out lines containing 'labels'
    lines = [line for line in lines if 'labels' not in line]
    # Write the filtered lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

# Apply the function to both text files
remove_labels_line(kbest_features_path)
remove_labels_line(selected_kbest_features_path)

print("Lines containing 'labels' have been removed from both files.")

selected_feature_names = selected_feature_names[selected_feature_names != 'labels']

print("'labels' has been removed from selected_feature_names")

In [11]:
df = df[selected_feature_names]

In [ ]:
output_path = "datasets/new/startup data.csv"

os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print(f"The DataFrame has been saved to {output_path}.")

# Numeric

In [ ]:
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif

df_original = pd.read_csv("datasets/kaggle/startup data.csv")

columns_to_drop = [
    'Unnamed: 6', 'closed_at', 'age_first_milestone_year', 'age_last_milestone_year', 
    'state_code.1', 'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate', 'is_software', 
    'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 
    'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 
    'is_othercategory', 'object_id', 'has_roundA', 'has_roundB', 
    'has_roundC', 'has_roundD'
]

original_columns = set(df_original.columns)
dropped_columns = set(columns_to_drop)
only_in_original = list(original_columns - dropped_columns)

features = [col for col in only_in_original if col not in ['status', 'labels']]
df_selected = df_original[features]
y = df_original['status']

label_encoders = {}
for col in df_selected.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_selected.loc[:, col] = le.fit_transform(df_selected[col].astype(str))
    label_encoders[col] = le

selector = SelectKBest(score_func=f_classif, k=10)
X_kbest = selector.fit_transform(df_selected, y)

selected_feature_names = df_selected.columns[selector.get_support(indices=True)]
selected_feature_scores = selector.scores_[selector.get_support(indices=True)]

top_features = pd.DataFrame({'Feature': selected_feature_names, 'Score': selected_feature_scores})
top_features = top_features.sort_values(by='Score', ascending=False)

directory = "Best Features/startup data"
os.makedirs(directory, exist_ok=True)

kbest_features_path = os.path.join(directory, "numeric_kbest_features.txt")
top_features.to_string(kbest_features_path, index=False)

print("Top Features Selected Using SelectKBest with Scores:")
print(top_features)
print(f"The top features have been saved to {kbest_features_path}")